In [28]:
from gensim.models import word2vec

In [29]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = doc2vec.Text8Corpus('text8')
word_model = word2vec.Word2Vec(sentences, size=600)

AttributeError: 'module' object has no attribute 'Text8Corpus'

In [6]:
# model.save('word2vec')

2017-05-29 03:10:09,787 : INFO : saving Word2Vec object under word2vec, separately None
2017-05-29 03:10:09,789 : INFO : not storing attribute syn0norm
2017-05-29 03:10:09,790 : INFO : storing np array 'syn0' to word2vec.wv.syn0.npy
2017-05-29 03:10:09,928 : INFO : storing np array 'syn1neg' to word2vec.syn1neg.npy
2017-05-29 03:10:10,021 : INFO : not storing attribute cum_table
2017-05-29 03:10:10,884 : INFO : saved word2vec


In [27]:
# word_model = word2vec.Word2Vec.load('word2vec')

2017-05-29 03:25:15,769 : INFO : loading Word2Vec object from word2vec
2017-05-29 03:25:16,140 : INFO : loading wv recursively from word2vec.wv.* with mmap=None
2017-05-29 03:25:16,141 : INFO : loading syn0 from word2vec.wv.syn0.npy with mmap=None
2017-05-29 03:25:16,227 : INFO : setting ignored attribute syn0norm to None
2017-05-29 03:25:16,228 : INFO : loading syn1neg from word2vec.syn1neg.npy with mmap=None
2017-05-29 03:25:16,368 : INFO : setting ignored attribute cum_table to None
2017-05-29 03:25:16,370 : INFO : loaded word2vec


In [19]:
def get_word_vector(sentence_tokens):
    vectors_available = [word for word in sentence_tokens if word in word_model.wv.vocab]
    result = np.zeros((word_model.vector_size))
    if len(vectors_available) > 0:
        result = np.mean([word_model[word] for word in vectors_available], axis=0)
    return result

In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
import numpy as np
%matplotlib inline
from random import Random
from texch.experiments import ClusteringExperiment, MultiClusteringExperiment
from texch.clustering.nltk import KMeansClusterer
from texch.preprocessing import PreprocessStep, Preprocessor
from texch.preprocessing.sklearn import TfidfVectorizer
from texch.clustering.nltk import KMeansClusterer

In [6]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(
    categories=[
        'alt.atheism', 'talk.religion.misc',
        'comp.graphics', 'sci.space'
    ],
    subset='test',
    random_state=42
)
labels = dataset.target
true_k = np.unique(labels).shape[0]

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

stopset = set(stopwords.words('english'))

data = []
for text in dataset.data:
    sent_tokens = []
    for sentence in sent_tokenize(text):
        sent = sentence.lower()
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(sent)
        sent_tokens += [w for w in tokens if not w in stopset]
    data.append(sent_tokens)

In [20]:
w2v_data = list(map(get_word_vector, data))

In [21]:
len(w2v_data[0])

600

In [10]:
from scipy.spatial.distance import euclidean

In [66]:
from texch.clustering.sklearn import KMeans

In [22]:
c = ClusteringExperiment(
    method=KMeansClusterer(
        true_k,
        distance=euclidean,
        rng=Random(15),
    ),
    preprocessor=Preprocessor([PreprocessStep(lambda x: x)])
)

In [23]:
c.set_input_data(w2v_data)

In [24]:
c.run()

2017-05-29 04:18:41,592 : INFO : Running experiment "
Preprocessor: PreprocessStep (id=1)
Method: NLTKClusterer:  (id=1)"...
2017-05-29 04:18:41,596 : INFO : Running preprocessing...
2017-05-29 04:18:41,599 : INFO : Step #0: PreprocessStep (id=1): finished in 7.15255737305e-06 sec
2017-05-29 04:18:41,600 : INFO : Finished preprocessing in 7.15255737305e-06
2017-05-29 04:18:41,602 : INFO : Running method...
2017-05-29 04:18:43,129 : INFO : Finished method in 1.52586698532 sec
2017-05-29 04:18:43,131 : INFO : Finished experiment in 1.52587413788 sec


,ExperimentID,ExperimentName,MethodSpent,PrepareFuncSpent,PreprocessorSpent,TotalSpent
0,1,\nPreprocessor: PreprocessStep (id=1)\nMethod:...,1.525867,0,0.000007,1.525874


In [25]:
c.set_true_labels(labels)
c.compute_scores()

,ExperimentID,ExperimentName,MethodSpent,PrepareFuncSpent,PreprocessorSpent,TotalSpent,entropy,homogeneity,v_measure,adj_rand_index,completeness,mutual_info_score,normalized_mutual_info_score,adjusted_mutual_info_score,fowlkes_mallows_score
0,1,\nPreprocessor: PreprocessStep (id=1)\nMethod:...,1.525867,0,0.000007,1.525874,1.385558,0.32269,0.320966,0.300971,0.319261,0.442354,0.320971,0.317618,0.478092


In [26]:
SCORES = [
    'homogeneity', 'completeness', 'v_measure',
    'adj_rand_index', 'adjusted_mutual_info_score',
    'fowlkes_mallows_score',
    'silhouette_coefficient', 'calinski_harabaz_score'
]

In [27]:
c.compute_scores(['silhouette_coefficient', 'calinski_harabaz_score'])
c.result[['ExperimentName'] + SCORES]

,ExperimentName,homogeneity,completeness,v_measure,adj_rand_index,adjusted_mutual_info_score,fowlkes_mallows_score,silhouette_coefficient,calinski_harabaz_score
0,\nPreprocessor: PreprocessStep (id=1)\nMethod:...,0.32269,0.319261,0.320966,0.300971,0.317618,0.478092,0.122395,179.93838


In [43]:
from sklearn.cluster import KMeans
k = KMeans(n_clusters=true_k)

In [44]:
k.fit(w2v_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [ ]:
w2v_data

In [47]:
from sklearn.metrics.cluster import homogeneity_completeness_v_measure
homogeneity_completeness_v_measure(labels, k.labels_)

ValueError: labels_true and labels_pred must have same size, got 1353 and 276586